# Monochromatic Neural Étendue Expansion Simulation Code

### This notebook can be used to produce the monochromatic étendue expanded simulation holograms shown in the manuscript and in the supplementary information.

### In the cells below please select one expander type and one target image. For example, to produce a 36x étendue expanded hologram with the neural étendue expander please select 'neural_mono_36x'. To produce a 4x étendue expanded hologram with a random expander [Kuo et al. 2020] please select 'random_4x'. The target images provided are labeled as '000.png', '001.png', and so on.

In [ ]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import random

from Expander import get_expander_phase
from SLM import get_slm_phase
from Render import get_intensity
from RI_wvl import RI_660, RI_517, RI_450, wvl_660, wvl_517, wvl_450
from Freq import butterworth, freq_filt
from Img import get_img

In [ ]:
### --- BEGIN CONFIG --- ###
# Choose only one of the following expanders.
#expander_type = 'random_4x'
#expander_type = 'random_16x'
#expander_type = 'random_36x'
#expander_type = 'random_64x'
#expander_type = 'neural_mono_4x'
#expander_type = 'neural_mono_16x'
#expander_type = 'neural_mono_36x'
expander_type = 'neural_mono_64x'

# Choose only one of the following target images.
#target_img_name = '000'
#target_img_name = '001'
target_img_name = '002'
### ---   END CONFIG --- ###

if '_4x' in expander_type:
    upsample_factor = 2
elif '_16x' in expander_type:
    upsample_factor = 4
elif '_36x' in expander_type:
    upsample_factor = 6
elif '_64x' in expander_type:
    upsample_factor = 8
else:
    assert('Undefined expander.')

In [ ]:
### --- BEGIN CONSTANTS --- ###
R = 512; C = 512
batch_size = 3
up_R = upsample_factor * R
up_C = upsample_factor * C
iterations = 1000; lr=0.1
### ---   END CONSTANTS --- ###

In [ ]:
### Set Random Seed for reproducibility
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")
set_seed(1996)

In [ ]:
### Define frequency filter ###
H = butterworth(R*upsample_factor,C*upsample_factor,upsample_factor)
H_torch = torch.from_numpy(H.astype(np.float32)); H_torch = H_torch.to('cuda:0')

### Optimize for Red Wavelength (660 nm)

In [ ]:
### Define target image (monochromatic at 660 nm) ###
img_name = os.path.join('Target_Images', target_img_name+'.png')
img, wvlpad_R, wvlpad_C = get_img(img_name, up_R, up_C, 'w')
target = torch.from_numpy(img.astype(np.float32)); target = target.to('cuda:0')
freq_target = freq_filt(target, H_torch)

In [ ]:
### Define expander (monochromatic at 660 nm) ###
expander_phase = get_expander_phase(expander_type, wvl_660, RI_660)
expander_phase = expander_phase.to('cuda:0')
expander_amp = torch.ones(R * upsample_factor, C * upsample_factor)
expander_amp = expander_amp.to('cuda:0')

In [ ]:
### Optimize for SLM ###
slm_amp = torch.ones(R, C); slm_amp = slm_amp.to('cuda:0')
slm_phase = get_slm_phase(freq_target, expander_amp, expander_phase, slm_amp, H_torch, \
                          batch_size, iterations, lr, R, C, upsample_factor, wvlpad_R, wvlpad_C)

In [ ]:
### Render etendue expanded hologram with optimized SLM pattern ###
slm_phase_upsample = F.interpolate(slm_phase, scale_factor=(upsample_factor,upsample_factor))
slm_amp_upsample = torch.ones_like(slm_phase_upsample)
loss, output = get_intensity(freq_target, expander_amp, expander_phase, slm_amp_upsample, slm_phase_upsample, H_torch, \
                             batch_size, R, C, upsample_factor, wvlpad_R, wvlpad_C)
print('Loss value is {}'.format(loss))

### Display Étendue Expanded Hologram and Target Image

In [ ]:
output_mono = torch.concat([output, output, output], axis=1)
output_mono = output_mono.permute((0,2,3,1))
output_mono = np.clip(output_mono[0,...].detach().cpu().numpy(), 0.0, 1.0)

plt.figure()
plt.title('Étendue Expanded Hologram')
plt.imshow(output_mono)

freq_target_mono = torch.stack([freq_target, freq_target, freq_target], axis=-1)
freq_target_mono = freq_target_mono.permute((0,1,2))
freq_target_mono = np.clip(freq_target_mono.detach().cpu().numpy(), 0.0, 1.0)

plt.figure()
plt.title('Target Image')
plt.imshow(freq_target_mono)